In [1]:
import os
import datetime
import math
import pandas as pd
from collections import Counter
from sym_creator import Symbol_Creator
#from sym_def import Symbol, Box, GFX, Font, Color, Pin, Property, PinName, mils_to_units, units_to_mils, side_dict, side_idx2val

In [2]:
out_dir = 'Created'
symbol_name = 'test_symbol'

In [3]:
# def symbol_from_count(symbol_name, count, pin_len = 300):
#     f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')
#     create_main_hdr(f)
    
#     w = estimate_width([str(i) for i in range(1, count + 1)])
#     h = estimate_height(count)
#     print('{}x{}'.format(w, h))
    
#     x = 0
#     y = 0
#     side = 'Left'
#     t_dict = {'Left': 'IN', 'Right': 'OUT'}
#     for i in range(1, count + 1):
#         add_pin(f, i, x, y, side, False, i, str(i), t_dict[side])
#         y += 100
#         if i >= count/2 and side=='Left':
#             side = 'Right'
#             y = 0
#             x += w + 2*pin_len
            
#     add_box(f, 300, -100, w, h)    
#     f.write('E')
#     f.close()

In [4]:
# symbol_from_count(symbol_name, 21)

# Create symbol from CSV

In [12]:
csv_file = 'import.csv'
csv = pd.read_csv(csv_file, delimiter=';')

In [13]:
creator = Symbol_Creator(out_dir=out_dir, out_symbol_name='export2')

In [14]:
csv

,Pin Order,Pin Label,Pin Number,Pin Type,Side,Inverted
0,L1,CLK_N,28,IN,2,True
1,L2,CLK_P,27,IN,2,False
2,R1,DB1_N,68,OUT,3,True
3,R2,DB1_P,69,OUT,3,False
4,R3,DB2_N,71,OUT,3,True
5,R4,DB2_P,72,OUT,3,False
6,R5,DB3_N,1,OUT,3,True
7,R6,DB3_P,2,OUT,3,False
8,R7,DA0_N,62,OUT,3,True
9,R8,DA0_P,61,OUT,3,False


In [15]:
test = csv.filter(['Pin Label', 'Pin Number', 'Pin Type'])

In [16]:
test_out = creator.predict_side_to_df(test, creator.default_cond_list)

In [17]:
csv['Side_Pred'] = test_out['Side'].values

In [18]:
csv

,Pin Order,Pin Label,Pin Number,Pin Type,Side,Inverted,Side_Pred
0,L1,CLK_N,28,IN,2,True,Left
1,L2,CLK_P,27,IN,2,False,Left
2,R1,DB1_N,68,OUT,3,True,Right
3,R2,DB1_P,69,OUT,3,False,Right
4,R3,DB2_N,71,OUT,3,True,Right
5,R4,DB2_P,72,OUT,3,False,Right
6,R5,DB3_N,1,OUT,3,True,Right
7,R6,DB3_P,2,OUT,3,False,Right
8,R7,DA0_N,62,OUT,3,True,Right
9,R8,DA0_P,61,OUT,3,False,Right


In [14]:
'' in 'test3'

True

In [6]:
# # TODO: SPI, I2C, etc.
# def get_bus_signals(df):
#     pass

In [7]:
creator = Symbol_Creator(out_dir=out_dir, out_symbol_name='export2')
creator.symbol_from_csv(csv_file, symbol_name)
creator.export_symbol()

900x1300.0


In [20]:
w = creator.estimate_box_width(list(csv['Pin Label'].values))
csv = creator.get_diff_pairs(csv)
test = creator.get_coordinates(csv, w)
test

,Pin Order,Pin Label,Pin Number,Pin Type,Side,Inverted,Diff,NC,PAD,GND,PWR,INV,sort,Duplicate,gap,x,y
0,L8,SEN_N,7,IN,2,True,-1,1,1,1,0,0,0.0,False,0.0,0,0.0
1,L13,SYNC_N,63,IN,2,True,-1,1,1,1,0,0,1.0,True,0.0,0,100.0
2,L7,SDIN,5,IN,2,False,-1,1,1,1,0,1,2.0,False,1.0,0,300.0
3,L6,SCLK,6,IN,2,False,-1,1,1,1,0,1,3.0,True,0.0,0,400.0
4,L9,VCM,22,ANALOG,2,False,-1,1,1,1,0,1,4.0,True,0.0,0,500.0
5,L5,RESET,48,IN,2,False,-1,1,1,1,0,1,5.0,True,0.0,0,600.0
6,L12,PDN/OVR,50,BI,2,False,-1,1,1,1,0,1,6.0,True,0.0,0,700.0
7,L1,CLK_N,28,IN,2,True,0,1,1,1,0,0,7.0,False,1.0,0,900.0
8,L2,CLK_P,27,IN,2,False,0,1,1,1,0,1,8.0,False,1.0,0,1100.0
9,L3,INA_N,41,ANALOG,2,True,8,1,1,1,0,0,9.0,False,1.0,0,1300.0


# Create csv

In [27]:
[v.endswith(('_N', '#')) for v in csv.loc[:, 'Pin Label'].values]

[True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False]

In [40]:
for lbl in csv['Pin Label'].values:
    print(any([v in lbl for v in ['GND', 'GROUND', 'PAD']]))

False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
True
False
False
False
False
False


In [41]:
csv

,Pin Order,Pin Label,Pin Number,Pin Type,Side,Inverted
0,R1,OUT1,19,POWER,3,False
1,R2,1P6V,11,ANALOG,3,False
2,R3,SNS,2,IN,3,False
3,R4,800mV,10,ANALOG,3,False
4,R5,OUT2,20,POWER,3,False
5,R6,FB,3,IN,3,False
6,L1,GND1,8,GROUND,2,False
7,L2,EN,14,IN,2,False
8,R7,400mV,9,ANALOG,3,False
9,L3,IN3,17,POWER,2,False


In [42]:
a = 'IN_3'
a[len('IN'):len('IN')+1]

'_'